# Esquema de Assinatura ElGamal

### Key generation

Para criar a chave pública e a chave privada, implementámos a função $\textit{keyGeneration}$ que tem como argumento um número de bits:

#### chave pública

$$PubKey = (p, r, b)$$

#### chave privada

$$PrivKey = (p,r,x)$$

onde,

<ul>
    <li> $\textbf{p}$ é um primo aleatório com o número de bits passado como argumento</li>
    <li> $\textbf{r}$ é uma raíz primitva</li>
    <li> $b = r^x \mod p $</li>
    <ul>
        <li>$\textbf{x}$ é um número aleatório no intervalo $1 \leq x \leq p-1$ e é primo relativo com $\textbf{p}$</li>
    </ul>
</ul>

In [1]:
import random
def keyGeneration(nbits):
    p = random_prime(2^(nbits//2),2^(nbits//2-1))
    
    
    r = primitive_root(p)
    
    Zp = IntegerModRing(p)
    
    r = Zp(r)
    
    while True:
        x = random.randint(1, p-1)
        if (gcd(x,p-1) == 1):
            break
            
    b = power_mod(r,x,p)
    
    PubKey = (p, r, b)
    PrivKey = (p, r, x)
    
    return PubKey, PrivKey

### Signature generation

Para gerar a assinatura criámos a função $\textit{signatureGeneration}$ que recebe como argumento a $\textbf{mensagem}$, a $\textbf{PubKey}$ e a $\textbf{PrivKey}$.

Começámos por gerar um $\textbf{k}$ aleatório que fosse $\textit{primo relativo}$ com $p-1$ e estivesse no intervalo $ 2 \leq k \leq p-2$.

De seguida gerámos um $\textbf{g}$:
$$ g = r^k$$
e o $\textbf{r}$ vem da $\textbf{PubKey}$.

De seguida, calculámos o $\textbf{s}$:
$$ s = (h - x * g) * l \mod (p-1)$$
onde, 
<ul>
    <li>$\textbf{h}$ é a mensagem após ser aplicada a função de $\textit{hash}$</li>
    <li>$\textbf{x}$ vem da $\textit{PrivKey}$</li>
    <li>$l = \frac{1}{k}$</li>
</ul>

Retornámos $\textbf{(g,s)}$.

In [2]:
import hashlib

def signatureGeneration(inputMessage, PubKey, PrivKey):
    p,r,_ = PubKey
    _,_,x = PrivKey
    
    global h
    
    HH =hashlib.sha256(str(inputMessage).encode()).hexdigest()
    h = ZZ('0x'+HH)
    
    while True:
        k = random.randint(2, p-2)
        if (gcd(k,p-1) == 1):
            break
            
    g = pow(r,k)
    
    
    Zp = IntegerModRing(p-1)
    l = 1/k
    s = Zp((h - x * int(g))*l)
    
    if s == 0:
        signatureGeneration(PubKey, PrivKey)
    
    
    return (g,s)


Criámos uma função de verificação para garantir que se obtinha o resultado desejado assim, fizemos duas verificações:
<ul>
    <li> se  $0 < r < p$ e se $0 < s < p-1$</li>
    <li>a assinatura só é válida se só se $r^h \equiv (b^g * g^s) (\mod p)$ </li>
</ul>

In [3]:
def verification(pair, m, PubKey):
    p,r,b = PubKey
    g,s = pair
    Zp = IntegerModRing(p)
    
    if 1 <= int(g) < p-1:
        v1 = Zp(pow(b,g) * pow(g,s))
        v2 = Zp(pow(r,h))
        
        if v1 == v2:
            return True
    return False

In [12]:
def main(bits, inputMessage):
    PubKey, PrivKey = keyGeneration(bits)
    pair = signatureGeneration(inputMessage, PubKey, PrivKey)
    result = verification(pair,h, PubKey)
    print("A assinatura é válida para a mensagem ", '"', inputMessage, '"?: ',result)

### Segurança



Este sistema criptográfico também tem as suas falhas, é possível, para alguém do exterior, descobrir o valor de $x$ presente na chave privada ou até, encontrar um colisão na função $\textit{hash}$, ou seja, $$ H(m) \equiv H(m')\: (mod\: p -1)$$ no entanto ambos os problemas são muito improváveis de acontecer.

Para evitar este tipo de ataques o dono da chave privada deve escolher um $k$ diferente para cada uma das suas assinaturas e ter a garantia que o valor atribuído a $k$ nunca será descoberto. Caso contrário, um $\textit{hacker}$ poderá ter acesso ao valor da sua chave privada, em particular, se o valor de $k$ for o mesmo em assinaturas diferentes, o $\textit{hacker}$ consegue computar o valor de $x$ diretamente.

### Exemplo 1

Os docentes da UC de Teoria de Números Computacional queriam comunicar entre si para trocar mensagens sobre as perguntas que iam ser colocadas no teste. Assim, cada professor criou uma chave pública e uma chave privada. Os alunos de LCC, com o conhecimento obtido na cadeira, tentaram decifrar as mensagens mas, como é óbvio, sem sucesso... 😭😭😳

In [15]:
@interact
def elGamal(bits=slider(300, 1028, 4, 8, 'Numero de bits', False), inputMessage=input_box('"Uma questão do teste podia ser sobre encriptar uma mensagem, usando o Esquema de Assinatura ElGamal"', label="$message$",width=100)):
    print("Bits: ", bits)
    main(bits, inputMessage)



Interactive function <function elGamal at 0x7f47d450d1f0> with 2 widgets
  bits: TransformIntSlider(value=300,…

### Exemplo 2

O presidente do Aljustrelinhos da Madeira, de seu nome António Figueiredo, queria corromper a arbitragem em Portugal. De forma a cumprir esta sua vontade, decidiu criar uma chave pública e uma chave privada para contornar a justiça nacional. As autoridades competentes decidiram contratar um hacker para o caso que infelizmente não conseguiu decifrar as mensagens visto que o António Figueiredo ainda continua em liberdade.

In [16]:
@interact
def elGamal(bits=slider(300, 1028, 4, 8, 'Numero de bits', False), inputMessage=input_box('"Vamos ter os padres que escolhemos e ordenámos, nas missas que celebramos, temos é de rezar e cantar bem"', label="$message$",width=100)):
    print("Bits: ", bits)
    main(bits, inputMessage)

Interactive function <function elGamal at 0x7f47d450d5e0> with 2 widgets
  bits: TransformIntSlider(value=300,…